In [1]:
Ruedas = {
    "base": {
         "secuencia": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
         "notch"    : "",
         "turnover" : ""
    },
    "I"  : {
         "secuencia": "EKMFLGDQVZNTOWYHXUSPAIBRCJ",
         "notch"    : "Q",
         "turnover" : "R"
    },
    "II" : {
         "secuencia": "AJDKSIRUXBLHWTMCQGZNPYFVOE",
         "notch"    : "E",
         "turnover" : "F"
    },
    "III": {
         "secuencia": "BDFHJLCPRTXVZNYEIWGAKMUSQO",
         "notch"    : "V",
         "turnover" : "W"
    },
    "IV" : {
         "secuencia": "ESOVPZJAYQUIRHXLNFTGKDCMWB",
         "notch"    : "J",
         "turnover" : "K"
    },
    "V"  : {
         "secuencia": "VZBRGITYUPSDNHLXAWMJQOFECK",
         "notch"    : "Z",
         "turnover" : "A"
    },
    "VI" : {
         "secuencia": "JPGVOUMFYQBENHZRDKASXLICTW",
         "notch"    : "ZM",
         "turnover" : "AN"
    },
    "VII": {
         "secuencia": "NZJHGRCXMYSWBOUFAIVLPEKQDT",
         "notch"    : "ZM",
         "turnover" : "AN"
    },
    "VIII": {
         "secuencia": "FKQHTLXOCBJSPDZRAMEWNIUYGV",
         "notch"    : "ZM",
         "turnover" : "AN"
    }
}

Ruedas_griegas = {
    "base" : {
         "secuencia": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
         "notch"    : "",
         "turnover" : ""
     },
    "beta" : {
         "secuencia": "LEYJVCNIXWPBQMDRTAKZGFUHOS",
         "notch"    : "",
         "turnover" : ""
     },
    "gamma": {
         "secuencia": "FSOKANUERHMBTIYCWLQPZXVGJD",
         "notch"    : "",
         "turnover" : ""
     }
}

Reflectores = {
    "base"  : "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "A"     : "EJMZALYXVBWFCRQUONTSPIKHGD",
    "B"     : "YRUHQSLDPXNGOKMIEBFZCWVJAT",
    "C"     : "FVPJIAOYEDRZXWGCTKUQSBNMHL",
    "B_thin": "ENKQAUYWJICOPBLMDXZVFTHRGS",
    "C_thin": "RDOBJNTKVEHMLFCWZAXGYIPSUQ"
}

In [2]:
class Rotor:
    def __init__(self, rueda, ringstellung: str):      
        """
        Rueda es el mapa de la parte inicial del fichero. 
        """

        self.base      = Ruedas["base"]["secuencia"]
        self.secuencia = rueda["secuencia"]
        self.notch     = rueda["notch"]
        self.turnover  = rueda["turnover"]

        # Aplicamos el ringstellung, moviendo cada letra ciertas posiciones con respecto a la base
        self.__ajusta_ringstellung__(ringstellung)


#
# ──────────────────────────────────────────────────────────── CONFIGURACIÓN ─────
#


    def ajusta_grundstellung(self, indice):
        self.base   = self.base[indice:] + self.base[:indice]
        self.secuencia = self.secuencia[indice:] + self.secuencia[:indice]
    
    
    def __ajusta_ringstellung__(self, ringstellung: str):
        indice_ring = Ruedas["base"]["secuencia"].find(ringstellung)
        self.secuencia = list(self.secuencia)
        
        for i in range(len(self.secuencia)):
            indice = Ruedas["base"]["secuencia"].find(self.secuencia[i]) + indice_ring
            self.secuencia[i] = Ruedas["base"]["secuencia"][indice % 26]
        
        cadena = ""
        for c in self.secuencia:
            cadena = cadena + c
        self.secuencia = cadena   
        self.secuencia = self.secuencia[-indice_ring:] + self.secuencia[:-indice_ring]


#
# ───────────────────────────────────────────────────────── ACCIÓN DEL ROTOR ─────
#
 
    def siguiente(self, indice: int):
        letra_a_buscar   = self.secuencia[indice]
        indice_siguiente = self.base.index(letra_a_buscar)
        return indice_siguiente


    def revertir(self, indice):
        letra_en_base    = self.base[indice]
        indice_de_salida = self.secuencia.index(letra_en_base)
        return indice_de_salida


    def rotar(self):
        self.base   = self.base[1:] + self.base[:1]
        self.secuencia = self.secuencia[1:] + self.secuencia[:1]


    def ha_tocado_notch(self):
        return self.base[0] in self.notch

In [3]:
class Reflector:
    def __init__(self, reflector: str):
        self.secuencia = reflector
    
    def reflejar(self, indice):
        """
        `Indice` representa la letra de base[indice].
        """
        letra_a_buscar = self.secuencia[indice]
        indice_siguiente = Reflectores["base"].find(letra_a_buscar)
        return indice_siguiente

In [4]:
class Plugboard:
    def __init__(self, parametros):
        """
        El parámetro `mapping` es una lista de duplas, de la forma
        ```
        [(letra1, letra2), (letra3, letra4)]
        ```
        
        De forma que se construirá un mapa tal que letra1 -> letra2, etc...
        """
        
        self.base    = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.mapping = {}

        # Nos guardamos primero un mapa de la forma A -> A, B -> B, ...
        for letra in self.base:
            self.mapping[letra] = letra

        # Y ahora lo sustituimos por lo que nos han mandado
        for dupla in parametros:
            self.mapping[dupla[0]] = dupla[1]
            self.mapping[dupla[1]] = dupla[0]
    
    
    def siguiente(self, letra: str) -> str:
        return self.mapping[letra]

In [5]:
class Enigma:
    def __init__(self, ruedas, ringstellungs: str, grundstellungs: str, reflector: str, steckern: list[(str, str)]):
        
        if len(ruedas) != len(ringstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del ringstellung")
        if len(ruedas) != len(grundstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del grundstellung")
        
        # Guardar la configuración inicial para el reseteo
        self.params = [ruedas, ringstellungs, grundstellungs, reflector, steckern]

        # Inicialización de los rotores        
        self.__setup_rotores__(ruedas, ringstellungs, grundstellungs)        

        # Montamos el resto de los componentes
        self.plugboard = Plugboard(steckern)
        self.reflector = Reflector(reflector)

        # Los rotores giran cuando se pulsa una letra de una manera determinada. 
        # La enigma M4 utilizó un cuarto rotor que no gira, por lo que el último campo se mantendrá a False. 
        # El primero siempre rota, por lo que constantemente estará a True
        # Utilizar esta estructura nos resultará especialmente útil cuando tengamos que controlar el double step. 
        self.needs_rotation = [False, False, False, True]


    def __setup_rotores__(self, ruedas, ringstellungs: str, grundstellungs: str):
        self.rotores = []
        for rotor, ring in zip(ruedas, ringstellungs):
            self.rotores.append(Rotor(rotor, ring))
        
        # Encontramos los índices correspondientes al grundstellung
        indices = []
        for letra in grundstellungs:
            indices.append(Ruedas["base"]["secuencia"].find(letra))

        for rotor, indice in zip(self.rotores, indices):
            rotor.ajusta_grundstellung(indice)
    
    
    def reset(self):
        self.__setup_rotores__(self.params[0], self.params[1], self.params[2])
        self.needs_rotation = [False, False, False, True]


#
# ─────────────────────────────────────────────────────────── FUNCIONAMIENTO ─────
#


    def __cifrar_caracter__(self, c: str):
        # ────────────────────────────────────────────────── ROTACION ─────
        #
        # Vigilar cuándo toca el notch, para girarlos cuando corresponda. 
        # Utilizar esta estructura nos resultará especialmente útil cuando tengamos que controlar el double step. 

        # Los rotores se iteran de derecha a izquierda:
        # <- [D, C, B, A] <- 
        self.needs_rotation[-2] = self.rotores[-1].ha_tocado_notch()
        self.needs_rotation[-3] = self.rotores[-2].ha_tocado_notch()
        
        # Single step
        for rotor, do_i_rotate in zip(self.rotores[::-1], self.needs_rotation[::-1]):
            if do_i_rotate:
                rotor.rotar()
        
        # Double step
        if self.needs_rotation[-3]:
            self.rotores[-2].rotar()


        # ─────────────────────────────────────────────────── CIFRADO ─────

        # El orden es el siguiente:
        # Pulsamos una tecla de entrada -> plugbooard -> rotores -> reflector -> rotores -> plugboard -> salida
        letra = self.plugboard.siguiente(c)

        # A partir de aquí, necesitamos los índices, que nos serán más cómodos. Retomaremos la letra más tarde
        indice_letra = Ruedas["base"]["secuencia"].find(letra)

        for rotor in reversed(self.rotores):
            indice_letra = rotor.siguiente(indice_letra)
        
        indice_letra = self.reflector.reflejar(indice_letra)
    
        for rotor in self.rotores:
            indice_letra = rotor.revertir(indice_letra)
        
        letra = Ruedas["base"]["secuencia"][indice_letra]

        letra = self.plugboard.siguiente(letra)
        
        return letra


    def cifrar(self, mensaje: str):        
        salida = ""

        for letra in mensaje:
            if letra == " ":
                salida = salida + " "
            else:
                salida = salida + self.__cifrar_caracter__(letra)
        
        return salida


In [6]:
print("Para probar la máquina Enigma, la configuraremos con los requisitos dados por el PDF, y comprobaremos que sale lo debido")

# Suponemos que utilizamos el orden [rotor de la derecha (VIII) -> rotor de la izquierda (beta)].
# Creemos que mantener este orden simplifica mucho el código, ya que no hay necesidades físicas como pasaba en la máquina original.
rotores   = [Ruedas_griegas["beta"], Ruedas["V"], Ruedas["VI"], Ruedas["VIII"]]
clavijas  = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]
reflector = Reflectores["C_thin"]

# Necesitamos revertir el orden del ring setting y la posición dados, pues se suponía en el PDF un funcionamiento idéntico al físico. 
ringstellungs            = "EPEL"   
grundstellungs_iniciales = "NAEM"

enigma = Enigma(rotores, ringstellungs, grundstellungs_iniciales, reflector, clavijas)

futuro_grundstellungs = "QEOB"
grundstellungs = enigma.cifrar(futuro_grundstellungs)
print(grundstellungs)

# Configuramos la nueva Enigma con el grundstellung que hemos obtenido
enigma = Enigma(rotores, ringstellungs, grundstellungs, reflector, clavijas)

mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"
# Quitar 2 primeros y 2 últimos grupos, pues se utilizaban para advertir fallos en la transmisión        
mensaje = mensaje[10:len(mensaje)-10]

mensaje_cifrado = enigma.cifrar(mensaje)

print(f"Mensaje original: {mensaje}\n")
print(f"Mensaje cifrado: {mensaje_cifrado}\n")

mensaje_museo = "KRKRALLEXXFOLGENDESISTSOFORTBEKANNTZUGEBENXXICHHABEFOLGELNBEBEFEHLERHALTENXXJANSTERLEDESBISHERIGXNREICHSMARSCHALLSJGOERINGJSETZTDERFUEHRERSIEYHVRRGRZSSADMIRALYALSSEINENNACHFOLGEREINXSCHRIFTLSCHEVOLLMACHTUNTERWEGSXABSOFORTSOLLENSIESAEMTLICHEMASSNAHMENVERFUEGENYDIESICHAUSDERGEGENWAERTIGENLAGEERGEBENXGEZXREICHSLEITEIKKTULPEKKJBORMANNJXXOBXDXMMMDURNHFKSTXKOMXADMXUUUBOOIEXKP"
mensaje_cifrado.replace(" ", "")
print(f'¿Son iguales el mensaje esperado y el que hemos obtenido? {mensaje_cifrado.replace(" ", "") == mensaje_museo}')


Para probar la máquina Enigma, la configuraremos con los requisitos dados por el PDF, y comprobaremos que sale lo debido
CDSZ
Mensaje original: LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW

Mensaje cifrado: KRKR ALLE XXFO LGEN DESI STSO FORT BEKA NNTZ UGEB ENXX ICHH ABEF OLGE LNBE BEFE HLER HALT ENXX JANS TERL EDES BISH ERIG XNRE ICHS MARS CHAL LSJG OERI NGJS ETZT DERF UEHR ERSI EYHV RRGR ZSSA DMIR ALYA LSSE INEN NACH FOLG EREI NXSC HRIF TLSC HEVO LLMA CHTU NTER WEGS XABS OFOR TSOL LENS IESA EMTL ICHE MASS NAHM ENVE RFUE GENY DIES ICHA USDE RGEG ENWA ERTI GENL AGEE RGEB ENX

In [7]:
# Se puede configurar la máquina para admitir 3 rotores en vez de 4
# Por ejemplo...
rotores  = [Ruedas["VIII"], Ruedas["VI"], Ruedas["V"]]
clavijas = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]
reflector = Reflectores["C"]

ringstellungs            = "LEP"   # 11 4 15 4
grundstellungs_iniciales = "MEA"   # 12 4 0 13

enigma = Enigma(rotores, ringstellungs, grundstellungs_iniciales, reflector, clavijas)
mensaje = "PRUEBA"
mensaje_cifrado = enigma.cifrar(mensaje)
enigma.reset()
mensaje_descifrado = enigma.cifrar(mensaje_cifrado)

mensaje, mensaje_cifrado, mensaje_descifrado

('PRUEBA', 'WSKUTF', 'PRUEBA')

# Ejercicio 4

El día 31 de octubre de 1944, el oficial Villegas desde tierra quiere enviarle al oficial Millán, que está en una expedición dentro de un submarino, un mensaje:

> 'Andres, los ingleses os van a meter un misilazo dentro de poco, tened cuidado'.

Que formateado como un texto a enviar por la enigma sería:

> J ANDRES Y LOS INGLESES OS VAN A METER UN MISILAZO DENTRO DE POCO Y TENED CUIDADO JX

Que en bloques de cuatro caracteres sería:

> JAND RESY LOSI NGLE SESO SVAN AMET ERUN MISI LAZO DENT RODE POCO YTEN EDCU IDAD OJX

El oficial Andrés tiene una configuración propia única y conocida solo por los oficiales:
* Zusatzwalze: Beta
* Walzenlage: 245
* Ringstellung: TAR
* Grundstellung: FIN

Por lo que el oficial Villegas codifica el mensaje con esa configuración, utilizando para el resto de parámetros los indicados por la fecha:

![](http://users.telenet.be/d.rijmenants/pics/wehrmachtkey-stab.jpg)

Esto es:

In [8]:
rotores  = [Ruedas["II"], Ruedas["IV"], Ruedas["V"]]
clavijas = [("K", "L"), ("I", "T"), ("F", "Q"), ("H", "Y"), ("X", "C"), ("N", "P"), ("V", "Z"), ("J", "B"), ("S", "B"), ("O", "G")]
reflector = Reflectores["C"]

ringstellungs            = "TAR"   
grundstellungs_iniciales = "FIN"

enigma1 = Enigma(rotores, ringstellungs, grundstellungs_iniciales, reflector, clavijas)

Ahora el oficial Villegas encripta su mensaje con esta configuración de enigma, 

In [9]:
mensaje = "JAND RESY LOSI NGLE SESO SVAN AMET ERUN MISI LAZO DENT RODE POCO YTEN EDCU IDAD OJX"
mensaje_destinatario = enigma1.cifrar(mensaje)
mensaje_destinatario

'BVPT QNTE NZLF HMRU FWUI XXHH BUXW BELV PQPR CUXV BPSS GNSO TXXW HRWT KWYO WBPQ QMY'

Y ahora se añade el destinatario al principio del mensaje:

In [10]:
mensaje_a_cifrar = "OFIC IALA NDRE SMIL LANX " + mensaje_destinatario
mensaje_a_cifrar

'OFIC IALA NDRE SMIL LANX BVPT QNTE NZLF HMRU FWUI XXHH BUXW BELV PQPR CUXV BPSS GNSO TXXW HRWT KWYO WBPQ QMY'

A continuación el oficial Villegas encripta el mensaje con la configuración de ese día y añade los caracteres de verificación, que por analogía con el ejercicio 2 seran "DUHF TETO"

In [11]:
rotores  = [Ruedas["IV"], Ruedas["V"], Ruedas["I"]]
clavijas = [("K", "L"), ("I", "T"), ("F", "Q"), ("H", "Y"), ("X", "C"), ("N", "P"), ("V", "Z"), ("J", "B"), ("S", "B"), ("O", "G")]
reflector = Reflectores["C"]

ringstellungs            = "VPQ"   
grundstellungs_iniciales = "JKM" 

verificacion = "DUHF TETO"

enigma2 = Enigma(rotores, ringstellungs, grundstellungs_iniciales, reflector, clavijas)
mensaje_enviado = verificacion + ' ' + enigma2.cifrar(mensaje_a_cifrar) + ' ' + verificacion

mensaje_enviado 

'DUHF TETO PITK NIEY BQEL BBZV ADCV ZYFC GEVK BFVH KCEZ IMCP RLBT QERM MSBT MPQP RGBE UDOR OIWS NSDH CDXB FNQW IEZE ILW DUHF TETO'

Y ese es el mensaje que se envía. El oficial de llaves recibe ese mensaje desde dentro del submarino, configura la enigma con las claves de ese día y desencripta el mensaje.

In [12]:
mensaje_recibido = mensaje_enviado[10:-10] 
print(mensaje_recibido)
enigma2.reset()
mensaje_desencriptado = enigma2.cifrar(mensaje_recibido)
mensaje_desencriptado

PITK NIEY BQEL BBZV ADCV ZYFC GEVK BFVH KCEZ IMCP RLBT QERM MSBT MPQP RGBE UDOR OIWS NSDH CDXB FNQW IEZE ILW


'OFIC IALA NDRE YMIL LANX BVPT QNTE QZLF HMRU FWUI XXHH BUXW BELV PQPR CUXV BPSS GNSO TXXW HRWC KWYO WBPQ QMY'

Y aquí el oficial de llaves ve que el mensaje está dirigido al oficial Andrés Millán, el cual con sus claves privadas desencripta el mensaje recibido:

In [13]:
mensaje_recibido, mensaje_desencriptado, mensaje_a_cifrar

('PITK NIEY BQEL BBZV ADCV ZYFC GEVK BFVH KCEZ IMCP RLBT QERM MSBT MPQP RGBE UDOR OIWS NSDH CDXB FNQW IEZE ILW',
 'OFIC IALA NDRE YMIL LANX BVPT QNTE QZLF HMRU FWUI XXHH BUXW BELV PQPR CUXV BPSS GNSO TXXW HRWC KWYO WBPQ QMY',
 'OFIC IALA NDRE SMIL LANX BVPT QNTE NZLF HMRU FWUI XXHH BUXW BELV PQPR CUXV BPSS GNSO TXXW HRWT KWYO WBPQ QMY')